**Backend (Flask)**

The backend will handle user authentication and CRUD operations for expenses. We'll use SQLite for simplicity.

1. Backend Setup
- Install the necessary libraries:

In [ ]:
pip install Flask Flask-SQLAlchemy Flask-JWT-Extended


2. Backend Code
- App Structure:

In [ ]:
backend/
│
├── app.py
├── models.py
├── database.db
├── requirements.txt
└── config.py


SyntaxError: invalid character '│' (U+2502) (<ipython-input-6-17096aac72b9>, line 2)

1. app.py: Main Flask Application

In [ ]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from datetime import timedelta

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['JWT_SECRET_KEY'] = 'supersecretkey'
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(hours=1)

db = SQLAlchemy(app)
jwt = JWTManager(app)

# Import models
from models import User, Expense

# Sign Up
@app.route('/signup', methods=['POST'])
def signup():
    data = request.get_json()
    new_user = User(username=data['username'], password=data['password'])
    db.session.add(new_user)
    db.session.commit()
    return jsonify({"message": "User created successfully!"}), 201

# Login
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    user = User.query.filter_by(username=data['username']).first()
    if user and user.password == data['password']:
        token = create_access_token(identity=user.id)
        return jsonify({"token": token}), 200
    return jsonify({"message": "Invalid credentials!"}), 401

# Add Expense
@app.route('/expenses', methods=['POST'])
@jwt_required()
def add_expense():
    user_id = get_jwt_identity()
    data = request.get_json()
    new_expense = Expense(user_id=user_id, category=data['category'], amount=data['amount'], comments=data.get('comments'))
    db.session.add(new_expense)
    db.session.commit()
    return jsonify({"message": "Expense added!"}), 201

# View Expenses
@app.route('/expenses', methods=['GET'])
@jwt_required()
def get_expenses():
    user_id = get_jwt_identity()
    expenses = Expense.query.filter_by(user_id=user_id).order_by(Expense.created_at.desc()).all()
    return jsonify([expense.to_dict() for expense in expenses]), 200

# Edit Expense
@app.route('/expenses/<int:id>', methods=['PUT'])
@jwt_required()
def edit_expense(id):
    data = request.get_json()
    expense = Expense.query.get(id)
    if expense:
        expense.category = data['category']
        expense.amount = data['amount']
        expense.comments = data.get('comments')
        db.session.commit()
        return jsonify({"message": "Expense updated!"}), 200
    return jsonify({"message": "Expense not found!"}), 404

# Delete Expense
@app.route('/expenses/<int:id>', methods=['DELETE'])
@jwt_required()
def delete_expense(id):
    expense = Expense.query.get(id)
    if expense:
        db.session.delete(expense)
        db.session.commit()
        return jsonify({"message": "Expense deleted!"}), 200
    return jsonify({"message": "Expense not found!"}), 404

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'models'

2. models.py: Database Models

In [ ]:
from app import db
from datetime import datetime

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(80), nullable=False)

class Expense(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    category = db.Column(db.String(80), nullable=False)
    amount = db.Column(db.Float, nullable=False)
    comments = db.Column(db.String(200), nullable=True)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

    def to_dict(self):
        return {
            'id': self.id,
            'category': self.category,
            'amount': self.amount,
            'comments': self.comments,
            'created_at': self.created_at
        }

db.create_all()


ModuleNotFoundError: No module named 'app'

**Frontend (HTML, CSS, JavaScript)**
- We’ll use simple HTML forms and JavaScript for the frontend. Authentication tokens are stored in localStorage to keep the user logged in across page reloads.

1. index.html: Expense Tracker Interface

Steps to Run the Application:
1. Backend:
- Initialize the database:

In [ ]:
python app.py


SyntaxError: invalid syntax (<ipython-input-10-b851947b46d7>, line 1)

Flask server will run on http://localhost:5000.

2. Frontend:
- Open index.html in the browser.
